pyJHTDB are failed to compile on windows. One alternative way might be to use [zeep](https://python-zeep.readthedocs.io/en/master/) package.    
More details can be found at [http://turbulence.pha.jhu.edu/service/turbulence.asmx](http://turbulence.pha.jhu.edu/service/turbulence.asmx)

In [1]:
import zeep
import numpy as np

client = zeep.Client('http://turbulence.pha.jhu.edu/service/turbulence.asmx?WSDL')
ArrayOfFloat = client.get_type('ns0:ArrayOfFloat')
ArrayOfArrayOfFloat = client.get_type('ns0:ArrayOfArrayOfFloat')
SpatialInterpolation = client.get_type('ns0:SpatialInterpolation')
TemporalInterpolation = client.get_type('ns0:TemporalInterpolation')

token="edu.jhu.pha.turbulence.testing-201406" #replace with your own token

nnp=5 #number of points
points=np.random.rand(nnp,3)

# convert to JHTDB structures
x_coor=ArrayOfFloat(points[:,0].tolist())
y_coor=ArrayOfFloat(points[:,1].tolist())
z_coor=ArrayOfFloat(points[:,2].tolist())
point=ArrayOfArrayOfFloat([x_coor,y_coor,z_coor]);

print(points)

[[0.69791304 0.18984549 0.98312756]
 [0.66030775 0.82447431 0.60665934]
 [0.44492412 0.16413926 0.78635297]
 [0.56623978 0.31035107 0.99200128]
 [0.60076763 0.45920464 0.85562809]]


#### In `GetData_Python`, Function_name could be  

GetVelocity,            GetMagneticField,          GetVectorPotential,    
GetVelocityGradient,    GetMagneticFieldGradient,  GetVectorPotentialGradient,    
GetVelocityHessian,     GetMagneticHessian,        GetVectorPotentialHessian,    
GetVelocityLaplacian,   GetMagneticFieldLaplacian, GetVectorPotentialLaplacian,    

GetPressure,            GetTemperature,            GetDensity,    
GetPressureGradient,    GetTemperatureGradient,    GetDensityGradient,    
GetPressureHessian,     GetTemperatureHessian,     GetDensityHessian,     

GetVelocityAndPressure, GetVelocityAndTemperature, GetForce, GetInvariant

In [2]:
Function_name="GetVelocityGradient" 
time=0.6
number_of_component=9 # change this based on function_name, see http://turbulence.pha.jhu.edu/webquery/query.aspx

result=client.service.GetData_Python(Function_name, token,"isotropic1024coarse", 0.6, 
                                     SpatialInterpolation("None_Fd4"), TemporalInterpolation("None"), point)
result=np.array(result).reshape((-1, number_of_component))

print(result)

[[  8.637329    -2.65904546   8.432873     1.28578365   6.64943171
   -3.37380672 -10.3590059   -2.05908656 -15.2024708 ]
 [ -0.47445306  -2.15972972  -0.07220937  -0.8424782   -3.14669132
    3.78339529  -1.62747037   3.04799342   3.60988474]
 [ -3.113433     4.86232471  -3.26763773  -1.73125684   2.799266
    3.09434485   2.610366     2.56994748   0.3923973 ]
 [  3.71506643  -2.088277    -4.363636     8.987988    -2.26683927
   11.9925947   -7.596671    -1.29600358  -1.69417107]
 [ -7.898257    -2.865736    -7.186355    -0.73715043   4.503267
    5.08899641   2.10948873 -10.1792564    3.12894464]]


#### In `GetPosition_Python`, Function_name could be
GetPosition only.

In [3]:
Function_name="GetPosition"
startTime=0.1
endTime=0.2
dt=0.02
number_of_component=3 # change this based on function_name, see http://turbulence.pha.jhu.edu/webquery/query.aspx

result=client.service.GetPosition_Python(Function_name, token,"isotropic1024coarse", startTime, endTime, dt, 
                                         SpatialInterpolation("None"), point)
result=np.array(result).reshape((-1, number_of_component))
print(result)

[[0.75016856 0.14140812 0.9664028 ]
 [0.6419222  0.8090803  0.5250584 ]
 [0.43126744 0.08140305 0.75407684]
 [0.6074615  0.28498182 0.9941733 ]
 [0.7184889  0.34636086 0.7491522 ]]


#### In `GetFilter_Python`, Function_name could be    

GetBoxFilter, GetBoxFilterSGSscalar, GetBoxFilterSGSvector,     
GetBoxFilterSGSsymtensor, GetBoxFilterSGStensor, GetBoxFilterGradient.

In [4]:
Function_name="GetBoxFilter" #could also be
field="u"
time=0.6
filterwidth=0.05
spacing=0 #spacing is only used in GetBoxFilterGradient, but always provide it.
number_of_component=3 # change this based on function_name, see http://turbulence.pha.jhu.edu/webquery/query.aspx

result=client.service.GetFilter_Python("GetBoxFilter",token,"isotropic1024coarse", field, 
                                       time, filterwidth, SpatialInterpolation("None"), point, spacing) 
result=np.array(result).reshape((-1, number_of_component))
print(result)

[[ 0.27206102 -1.30481255  0.06045016]
 [-0.12582369 -0.37959147 -1.10447919]
 [-0.8601785  -0.65143985 -0.8783631 ]
 [-0.48609233 -0.95902133 -0.22834729]
 [-0.42911473 -0.80377346  0.25465983]]


In [5]:
import struct
import base64

field="u"
timestep=1
x_start=1
y_start=1
z_start=1
x_end=2
y_end=5
z_end=8
x_step=1
y_step=1
z_step=1
filter_width=0
    
result=client.service.GetAnyCutoutWeb(token,"isotropic1024coarse", field, timestep,
                                       x_start, y_start, z_start, x_end, y_end, z_end,
                                      x_step, y_step, z_step, filter_width, "")  # put empty string for the last parameter

# transfer base64 format to numpy
number_of_component=3 # change this based on the field
nx=len(range(x_start, x_end+1, x_step))
ny=len(range(y_start, y_end+1, y_step))
nz=len(range(z_start, z_end+1, z_step))
base64_len=int(nx*ny*nz*number_of_component)
base64_format='<'+str(base64_len)+'f'

result=struct.unpack(base64_format, result)
result=np.array(result).reshape((nz, ny, nx, number_of_component))
print(result.shape)  # see the shape of the result and compare it with nx, ny, nz and number of component

(20, 20, 20, 3)
